In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input

import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2023-04-23 18:28:12.389609: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-23 18:28:12.436489: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-23 18:28:13.209322: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Directory paths for training, validation, and test data
train_dir = 'train'  # path to the directory containing training images

In [3]:
datagen = ImageDataGenerator()

In [4]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # resize images to (224, 224) as required by ResNet-50
    batch_size=32,

    class_mode='categorical'  # set to 'categorical' for multi-class classification
)

Found 22 images belonging to 4 classes.


In [5]:
train_generator.class_indices

{'falafel': 0, 'pizza': 1, 'shawarma': 2, 'spoon': 3}

In [6]:
# vals_generator = 

In [7]:
# Load the pretrained ResNet-50 model without the top (fully connected) layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model to prevent its weights from being updated during training
base_model.trainable = False


2023-04-23 18:28:47.544717: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [8]:
# class_labels = decode_predictions(np.eye(1, 1000, 1), top=1000)[0]
# class_labels = [label[1] for label in class_labels]
# print("ImageNet classes: ", class_labels)

In [10]:
# Number of classes in your dataset, including pizza, shawarma, and ImageNet classes
num_classes = 4
# Add custom output layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [11]:
# Create final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

In [12]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
# model.summary()

In [14]:
model.fit(
train_generator,
steps_per_epoch=len(train_generator),
epochs=10,
validation_data=None) #

Epoch 1/10


2023-04-23 18:28:56.480944: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


1/1 [==============================] - 4s 4s/step - loss: 1.9763 - accuracy: 0.1364
Epoch 2/10
1/1 [==============================] - 1s 887ms/step - loss: 0.5436 - accuracy: 0.7727
Epoch 3/10
1/1 [==============================] - 1s 873ms/step - loss: 0.0958 - accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 1s 883ms/step - loss: 0.0234 - accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 1s 882ms/step - loss: 0.0083 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 1s 894ms/step - loss: 0.0037 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 1s 894ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 1s 880ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 1s 876ms/step - loss: 6.3250e-04 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 1s 910ms/step - loss: 4.1228e-04 - accuracy: 1.0000


In [17]:


# Load a test image
# image_path = 'train/shawarma/shawarma_7.png'
# image_path = 'train/pizza/pizza_5.png'
# image_path = 'train/falafel/falafel_6.png'
# image_path = 'train/spoon/spoon_3.png'


image = load_img(image_path, target_size=(224, 224))

# Convert the image to a numpy array
image_array = img_to_array(image)

# Preprocess the image
image_array = preprocess_input(image_array)
image_array = image_array.reshape((1,) + image_array.shape)

# Make prediction
prediction = model.predict(image_array)
class_names = ['falafel','pizza','shawarma','spoon']
# Get the class label with highest probability
class_index = prediction.argmax()
class_label = class_names[class_index]  # assuming class_names is a list of class labels

# Get the probability of the predicted class
probability = prediction[0, class_index]

print(class_label)
# Print the predicted class label and probability
print(f'Predicted Class:  Probability: {probability:.2f}')



1/1 [==============================] - 0s 106ms/step
pizza
Predicted Class:  Probability: 1.00


In [19]:
model.save('resnet50_model.h5')